In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
Extrai o dataset SPIE-AAPM Lung CT Challenge do drive. O conjunto de imagens pode ser acessado no link abaixo:
  - https://wiki.cancerimagingarchive.net/display/Public/SPIE-AAPM+Lung+CT+Challenge#19039197a19462154cc74bea92039089e61a0f44
"""

!unzip -q /content/drive/MyDrive/TCC/dataset.zip

In [ ]:
"""
O código abaixo realiza as seguintes operações:
  - Redimensiona as imagens para um tamanho especificado;
  - Define um tamanho de lote para o treinamento do modelo;
  - Divide as imagens em conjuntos de treinamento e validação;
  - Imprime o nome das classes do dataset.
"""

IMAGE_SIZE = (180,180)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    validation_split=0.2,
    subset='training',
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    validation_split=0.2,
    subset='validation',
    seed=1337,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

train_ds.class_names

In [ ]:
"""
Esse código cria uma figura com nove subplots e plota nove imagens aleatórias do conjunto
de treinamento do modelo - que contém tumores benignos (0) e malignos (1).
"""

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(int(labels[i]))
        plt.axis('off')

In [ ]:
"""
Realiza o processo de data augmentation nas imagens:
  - Aplica uma operação de espelhamento horizontal aleatório nas imagens de entrada;
  - Aplica uma rotação nas imagens de entrada com uma amplitude de até 0.1 radianos.
"""

from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
])

In [ ]:
"""
Exibe 10 imagens do dataset após o processo de data augmentation.
"""

plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
"""
Permite que o conjunto de dados carregue 32 lotes de dados em segundo plano enquanto
o modelo está sendo treinado com o lote anterior.
"""

BUFFER_SIZE=32

train_ds = train_ds.prefetch(buffer_size=BUFFER_SIZE)
val_ds = val_ds.prefetch(buffer_size=BUFFER_SIZE)

In [ ]:
"""
Constrói o modelo baseado na arquitetura Vgg19
"""

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = layers.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    previous_block_activation = x

    for size in [128, 256, 512, 728]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

        residual = layers.Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)

        x = layers.add([x, residual])

        previous_block_activation = x

    x = layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(units=1, activation='sigmoid')(x)

    return keras.Model(inputs, outputs)


model = make_model(input_shape=IMAGE_SIZE + (3,), num_classes=2)

keras.utils.plot_model(model, show_shapes=True)

In [ ]:
"""
Executa o treinamento do modelo Vgg19
"""

EPOCHS=50

callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]

model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_ds, epochs=EPOCHS, callbacks=callbacks, validation_data=val_ds)

In [ ]:
"""
Método que utiliza o modelo treinado para classificar uma imagem
"""

def classify_image(image_path):
  img = keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  predictions = model.predict(img_array)
  score = predictions[0]
  benign = 100 * (1 - score)
  malignant = 100 * score

  print("This image is %.2f%% benign and %.2f%% malignant." % (benign, malignant))

In [ ]:
"""
Testa o modelo classificando uma imagem com tumor benigno
"""

classify_image('dataset/benign/011-001.jpg')

In [ ]:
"""
Testa o modelo classificando uma imagem com tumor maligno
"""

classify_image('dataset/malignant/011-006.jpg')